In [ ]:
!pip install langchain
!pip install chromadb
!pip install openai
!pip install pypdf
!pip install tiktoken
!pip install ragas

##RAG using Agent_toolkits by langchain

In [ ]:
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo
)
import os
os.environ['OPENAI_API_KEY'] = 'key'

llm = OpenAI(temperature=0.1, verbose=True)
embeddings = OpenAIEmbeddings()
loader = PyPDFLoader('/content/infosys-ar-23.pdf')
pages = loader.load_and_split()
store = Chroma.from_documents(pages, embeddings, collection_name='annualreport')

# Create vectorstore info object - metadata repo?
vectorstore_info = VectorStoreInfo(
    name="annual_report",
    description="a banking annual report as a pdf",
    vectorstore=store
)

toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)

agent_executor = create_vectorstore_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)



In [ ]:
#save db
# store = Chroma.from_documents(pages, embeddings,collection_name='annualreport', persist_directory="./chroma_db")

#load
store = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)


##Evaluation

In [ ]:
from ragas.metrics import faithfulness, answer_relevancy, context_relevancy, context_recall
from ragas.langchain import RagasEvaluatorChain
from langchain.docstore.document import Document

eval_chains = {
    m.name: RagasEvaluatorChain(metric=m)
    for m in [faithfulness, answer_relevancy, context_relevancy]
}

### Testing our RAG

In [ ]:
query = 'What is Total comprehensive income for the year attributable to the owners of the Company, Standalone - year 2023?'
response = agent_executor.run(query)
print(response)



> Entering new AgentExecutor chain...
 I need to find the answer in the annual report
Action: annual_report
Action Input: Total comprehensive income for the year attributable to the owners of the Company, Standalone - year 2023
Observation:  24,000
Thought: I now know the final answer
Final Answer: 24,000

> Finished chain.
24,000


### Evaluating RAG

In [ ]:
doc = Document(page_content=chunk.page_content)

for name, eval_chain in eval_chains.items():
    score_name = f"{name}_score"
    inputs = {
        'query': query,
        'source_documents': [doc],
        'result': response
    }
    score = eval_chain(inputs)[score_name]
    print(f"{score_name}: {score}")

faithfulness_score: 0.0
answer_relevancy_score: 0.7575130488250109
context_relevancy_score: 0.02564102564102564


In [ ]:
query = "What is the Date of acquisition of Blue Acorn iCi Inc?"
response = agent_executor.run(query)
search = store.similarity_search_with_score(query)
chunk = search[0][0]

doc = Document(page_content=chunk.page_content)

for name, eval_chain in eval_chains.items():
    score_name = f"{name}_score"
    inputs = {
        'query': query,
        'source_documents': [doc],
        'result': response
    }
    score = eval_chain(inputs)[score_name]
    print(f"{score_name}: {score}")



> Entering new AgentExecutor chain...
 I need to find the date of acquisition of Blue Acorn iCi Inc.
Action: annual_report
Action Input: Date of acquisition of Blue Acorn iCi Inc
Observation:  October 27, 2020
Thought: I now know the final answer
Final Answer: October 27, 2020

> Finished chain.
faithfulness_score: 0.0
answer_relevancy_score: 0.764300869513152
context_relevancy_score: 0.03614457831325301


In [ ]:

query = "What initiavtive company took for women and Women in Tech particular?"
response = agent_executor.run(query)
search = store.similarity_search_with_score(query)
chunk = search[0][0]

doc = Document(page_content=chunk.page_content)

for name, eval_chain in eval_chains.items():
    score_name = f"{name}_score"
    inputs = {
        'query': query,
        'source_documents': [doc],
        'result': response
    }
    score = eval_chain(inputs)[score_name]
    print(f"{score_name}: {score}")



> Entering new AgentExecutor chain...
 I need to find a source that has information about initiatives taken by the company for women and Women in Tech
Action: annual_report_with_sources
Action Input: initiatives taken by the company for women and Women in Tech
Observation: {"answer": " The company has taken initiatives such as the IamtheFuture Women in Leadership Program, Employee Resource Groups, Anti-Sexual Harassment Initiative, and Orbit Next to support women and Women in Tech.\n", "sources": "/content/infosys-ar-23.pdf"}
Thought: I now know the final answer
Final Answer: The company has taken initiatives such as the IamtheFuture Women in Leadership Program, Employee Resource Groups, Anti-Sexual Harassment Initiative, and Orbit Next to support women and Women in Tech.

> Finished chain.
faithfulness_score: 0.5
answer_relevancy_score: 0.9430522315049544
context_relevancy_score: 0.047619047619047616


In [ ]:
query = "What is the process of Selection and appointment of new directors"
response = agent_executor.run(query)
search = store.similarity_search_with_score(query)
chunk = search[0][0]

doc = Document(page_content=chunk.page_content)

for name, eval_chain in eval_chains.items():
    score_name = f"{name}_score"
    inputs = {
        'query': query,
        'source_documents': [doc],
        'result': response
    }
    score = eval_chain(inputs)[score_name]
    print(f"{score_name}: {score}")



> Entering new AgentExecutor chain...
 I need to find information about the selection and appointment of new directors
Action: annual_report
Action Input: selection and appointment of new directors
Observation:  The Board delegates the screening and selection process to the Nomination and Remuneration Committee, which consists exclusively of independent directors. The Committee, based on defined criteria, as laid out in the Nomination and Remuneration Policy, presents a diverse slate of recommendations of eligible candidates to the Board. The Board recommends the appointment of the director to the shareholders. The proposal is placed before the shareholders for approval.
Thought: I now know the final answer
Final Answer: The Board delegates the screening and selection process to the Nomination and Remuneration Committee, which consists exclusively of independent directors. The Committee, based on defined criteria, as laid out in the Nomination and Remuneration Policy, presents a dive